In [1]:
# import modules
%matplotlib notebook

import numpy as np
from  multiprocessing import Pool

import matplotlib.pyplot as plt
import matplotlib
from cycler import cycler
_colc = matplotlib.rcParams['axes.prop_cycle'].concat(cycler(color=['blue', 'black']))
matplotlib.rcParams['font.size'] = 12
matplotlib.rcParams['xtick.direction'] = 'in'
matplotlib.rcParams['ytick.direction'] = 'in'

In [2]:
%%time

import chempl
from chempl import (cs, run_one_model, get_total_charge,
                    get_phy_params_default, N_H_to_Av, Av_to_N_H,
                    N_H_to_ngas, Td_from_Av, chem2tex, printFormationDestruction)

CPU times: user 4.69 ms, sys: 7.09 ms, total: 11.8 ms
Wall time: 890 ms


# define the solver

Here it is called "model".  It acts as a solver.

In [3]:
%%time

model = chempl.ChemModel(
    fReactions=b'rate12_noSurf_only_H2_formation.dat',
    fInitialAbundances=b'initial_abundances.dat').prepare().set_solver()

CPU times: user 168 ms, sys: 4.02 ms, total: 172 ms
Wall time: 180 ms


# Load the default physical parameters

In [4]:
phy_params = chempl.get_phy_params_default()

# A simple single run

In [5]:
%%time

t0 = 0.0
dt = 1.0
t_ratio = 1.1
tmax_yr = 1e6
nmax = 2000

res = chempl.run_one_model(
    p={'model_id': 0,
       'y0': model.abundances,
       't0': t0,
       'dt0': dt,
       't_ratio': t_ratio,
       'tmax': tmax_yr*cs.phy_SecondsPerYear,
       'nmax': nmax,
       'phy_params': phy_params
      },
    model=model,
    verbose=False
)

CPU times: user 2.76 s, sys: 19.8 ms, total: 2.78 s
Wall time: 2.77 s


# Define a list of different physical parameter combinations

In [6]:
n_s = np.logspace(3,5,num=16)
phy_params_s = [{b'n_gas': n, b'T_gas': 20.0} for n in n_s]

# Run multiple models one by one

In [7]:
%%time
res0 = chempl.run_multiple_params_sequentially(
    model=model, phy_params_s=phy_params_s, t0=t0, dt0=dt, t_ratio=t_ratio, tmax_yr=tmax_yr, nmax=2000)

CPU times: user 39.4 s, sys: 199 ms, total: 39.6 s
Wall time: 39.4 s


# The same, but ...

using the final state of the previous one as the initial state of the next

In [8]:
%%time
res0 = chempl.run_multiple_params_sequentially(follow=True,
    model=model, phy_params_s=phy_params_s, t0=t0, dt0=dt, t_ratio=t_ratio, tmax_yr=tmax_yr, nmax=2000)

CPU times: user 13.2 s, sys: 92.6 ms, total: 13.3 s
Wall time: 13.2 s


# Try with multiprocessing

In [9]:
def worker(p):
    # Note that the "model" object must be global
    t = p['t0']
    dt = p['dt']
    t_ratio = p['t_ratio']
    nMax = p['nmax']
    tMax = p['tmax']
    
    y = [_ for _ in p['y0']]
    store = {'ts': [], 'ys': []}
    model.set_solver(model_id=p['model_id'])
    model.set_phy_params_by_dict(p['phy_params'])
    for i in range(nMax):
        t, y = model.update(y, t=t, dt=dt, verbose=p.get('verbose'))
        store['ts'].append(t)
        store['ys'].append(y)
        if t >= tMax:
            break
        dt = dt * t_ratio
        if t+dt > tMax:
            dt = tMax - t
    return store

In [10]:
from multiprocessing import Pool
pool = Pool(4)

In [11]:
%%time

p_s = [{'model_id': i,
        'y0': model.abundances,
        't0': t0,
        'dt': dt,
        't_ratio': t_ratio,
        'tmax': tmax_yr*cs.phy_SecondsPerYear,
        'nmax': nmax,
        'phy_params': pps,
        'verbose': False,
      } for (i,pps) in enumerate(phy_params_s)]

res1 = pool.map(worker, p_s)

CPU times: user 143 ms, sys: 64.6 ms, total: 207 ms
Wall time: 13 s


In [12]:
# The results should be the same from sequential or parallel runs

[np.any(np.array(s0['ys'])-np.array(s1['ys'])) for s0,s1 in zip(res0, res1)]

[False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]